In [1]:
import json

import numpy as np

import pandas as pd

from IPython.display import display

In [2]:
# Configurazione opzioni Pandas

pd.set_option('display.max_columns', None)

In [3]:
# Definizione funzioni ausiliarie

def float_converter(s):
    s = s.replace(",", ".")
    return s

def percentage_calculator(row):
    if(row['espressi'] == 0):
        return 0
    else:
        return row['voti']/row['espressi']

In [4]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_vaccinazioni_covid = config_data['path_covid_19_france_vaccinations_age_sexe_dep']
file_path_andamento_dei_prezzi_al_consumo = config_data['path_economicref_france_indice_prix_consommation']
file_path_risultati_delle_elezioni_legislative_1_turno = config_data['path_elections_france_legislatives_2022_1er_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_legislative_2_turno = config_data['path_elections_france_legislatives_2022_2nd_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_presidenziali_1_turno = config_data['path_elections_france_presidentielles_2022_1er_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_presidenziali_2_turno = config_data['path_elections_france_presidentielles_2022_2nd_tour_par_bureau_de_vote']
file_path_dati_sulla_disoccupazione = config_data['path_labouref_france_departement_quarter_jobseeker']
file_path_andamento_dei_prezzi_dei_carburanti = config_data['path_prix_des_carburants_j_1_001']

file_path_risultati_elezioni_presidenziali_1_turno = config_data['path_risultati_elezioni_presidenziali_1_turno']
file_path_risultati_elezioni_presidenziali_2_turno = config_data['path_risultati_elezioni_presidenziali_2_turno']

In [5]:
# Caricamento e cleaning dataset vaccinazioni_covid

df_vaccinazioni_covid = pd.read_csv(file_path_vaccinazioni_covid, sep=';')
df_vaccinazioni_covid.rename(columns={"Date": "data", "Nombre de doses n°1": "numero_prime_dosi", "Nombre cumulé de doses n°1":"numero_cumulativo_prime_dosi", "Type de variable":"tipo_variabile","Valeur de la variable":"valore_variabile","Libellé de la variable":"etichetta_variabile", "Nom Officiel Département":"nome_dipartimento", "Code Officiel Région":"codice_regione", "Nom Officiel Région":"nome_regione", "Code Officiel Département":"codice_dipartimento", "Nombre de vaccinations complètes (doses n°2)":"numero_seconde_dosi", "Nombre de personnes vaccinées avec une dose de rappel (3e dose)":"numero_terze_dosi", "Nombre de personnes vaccinées avec une deuxième dose de rappel (4e dose)":"numero_quarte_dosi", "Nombre de doses de rappel bivalent":"numero_dosi_bivalenti", "Nombre de personnes ayant reçu une troisième dose de rappel":"numero_persone_terza_dose", "Nombre cumulé de vaccinations complètes (doses n°2)":"numero_cumulativo_seconde dosi", "Nombre cumulé de personnes vaccinées avec une dose de rappel (3e dose)":"numero_cumulativo_terze_dosi", "Nombre cumulé de personnes vaccinées avec une deuxième dose de rappel (4e dose)":"numero_cumulativo_quarte_dosi", "Nombre cumulé de personnes vaccinées avec une dose de rappel bivalent":"numero_cumulativo_dosi_bivalenti", "Nombre cumulé de personnes ayant reçu une troisième dose de rappel":"numero_cumulativo_persone_terza_dose_richiamo", "Couverture vaccinale partielle (dose n°1)":"copertura_vaccinale_prima_dose", "Couverture vaccinale complète (dose n°2)":"copertura_vaccinale_seconda_dose", "Couverture vaccinale rappel/3e dose":"copertura_vaccinale_terza_dose", "Couverture vaccinale 2e rappel/4e dose":"copertura_vaccinale_quarta_dose", "Couverture vaccinale du rappel bivalent":"copertura_vaccinale_dose_bivalente", "Couverture vaccinale de la troisième dose de rappel":"copertura_vaccinale_terza_dose_richiamo"}, inplace=True)
df_vaccinazioni_covid.drop(['numero_persone_terza_dose','numero_cumulativo_persone_terza_dose_richiamo', 'copertura_vaccinale_terza_dose_richiamo'], axis=1, inplace=True)
new_ordered_columns = ["data","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","tipo_variabile","etichetta_variabile","valore_variabile","numero_prime_dosi","numero_seconde_dosi","numero_terze_dosi","numero_quarte_dosi","numero_cumulativo_prime_dosi","numero_cumulativo_seconde dosi","numero_cumulativo_terze_dosi","numero_cumulativo_quarte_dosi","numero_dosi_bivalenti","numero_cumulativo_dosi_bivalenti", "copertura_vaccinale_prima_dose", "copertura_vaccinale_seconda_dose", "copertura_vaccinale_terza_dose", "copertura_vaccinale_quarta_dose", "copertura_vaccinale_dose_bivalente"]
df_vaccinazioni_covid = df_vaccinazioni_covid.reindex(columns=new_ordered_columns)
#df_vaccinazioni_covid.head(10)

In [6]:
# Caricamento e cleaning dataset andamento_prezzi_consumo

df_andamento_prezzi_consumo = pd.read_csv(file_path_andamento_dei_prezzi_al_consumo, sep=';')
df_andamento_prezzi_consumo.rename(columns={"Période" : "periodo", "Indice" : "suggerimento", "Valeur": "valore","Code" : "codice","Type" : "tipo","Sous-type" : "sottotipo", "Base" : "base", "Population" : "popolazione", "Zone concernée" : "zona_interessata", "Code COICOP" : "codice_COICOP", "Désignation COICOP" : "designazione_COICOP", "Série arrêtée" : "serie_interrotta", "Référence INSEE" : "riferimento_INSEE"}, inplace=True)
#df_andamento_prezzi_consumo.head(10)

In [7]:
# Caricamento e cleaning dataset risultati_elezioni_legislative_1_turno

df_risultati_elezioni_legislative_1_turno = pd.read_csv(file_path_risultati_delle_elezioni_legislative_1_turno, sep=';', dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_legislative_1_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_della_circoscrizione", "Libellé de la circonscription" : "nome_della_circoscrizione", "Code de la commune" : "codice_del_comune", "Libellé de la commune" : "nome_del_comune", "Code du b.vote" : "codice_del_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "tendenza_politica", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/iscritti", "Code Officiel EPCI" : "codice_ufficiale_EPCI", "Nom Officiel EPCI" : "nome_ufficiale_dello_EPCI", "Code Officiel Région" : "codice_ufficiale_regionale", "Nom Officiel Région" : "nome_ufficiale_regionale", "location" : "coordinate", "lib_du_b_vote" : "etichetta_del_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
#df_risultati_elezioni_legislative_1_turno.head(10)

In [8]:
# Caricamento e cleaning dataset risultati_elezioni_legislative_2_turno

df_risultati_elezioni_legislative_2_turno = pd.read_csv(file_path_risultati_delle_elezioni_legislative_2_turno, sep=';', dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_legislative_2_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_della_circoscrizione", "Libellé de la circonscription" : "nome_della_circoscrizione", "Code de la commune" : "codice_del_comune", "Libellé de la commune" : "nome_del_comune", "Code du b.vote" : "codice_del_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "tendenza_politica", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/iscritti", "Code Officiel EPCI" : "codice_ufficiale_EPCI", "Nom Officiel EPCI" : "nome_ufficiale_dello_EPCI", "Code Officiel Région" : "codice_ufficiale_regionale", "Nom Officiel Région" : "nome_ufficiale_regionale", "location" : "coordinate", "lib_du_b_vote" : "etichetta_del_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
#df_risultati_elezioni_legislative_2_turno.head(10)

In [9]:
# Caricamento e cleaning dataset risultati_elezioni_presidenziali_1_turno

df_risultati_elezioni_presidenziali_1_turno = pd.read_csv(file_path_risultati_delle_elezioni_presidenziali_1_turno, sep=';', dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_presidenziali_1_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_circoscrizione", "Libellé de la circonscription" : "nome_circoscrizione", "Code de la commune" : "codice_comune", "Libellé de la commune" : "nome_comune", "Code du b.vote" : "codice_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "tendenza_politica", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/espressi", "Code Officiel EPCI" : "codice_EPCI", "Nom Officiel EPCI" : "nome_EPCI", "Code Officiel Région" : "codice_regione", "Nom Officiel Région" : "nome_regione", "location" : "coordinate", "lib_du_b_vote" : "etichetta_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
new_ordered_columns = ["codice_regione","nome_regione","codice_dipartimento","nome_dipartimento","codice_EPCI", "nome_EPCI","codice_comune","nome_comune","codice_circoscrizione","nome_circoscrizione","codice_seggio_elettorale","iscritti","astenuti","% astenuti/iscritti","votanti","% votanti/iscritti","bianche","% bianche/iscritti","% bianche/votanti","nulle","% nulle/iscritti","% nulle/votanti","espressi","% espressi/iscritti","% espressi/votanti","pannello_numero","genere","cognome","nome","tendenza_politica","voti","% voti/iscritti","% voti/espressi","coordinate","etichetta_seggio_elettorale","codice_scrutinio"]
df_risultati_elezioni_presidenziali_1_turno = df_risultati_elezioni_presidenziali_1_turno.reindex(columns=new_ordered_columns)
df_risultati_elezioni_presidenziali_1_turno['% voti/espressi'] = df_risultati_elezioni_presidenziali_1_turno['% voti/espressi'].map(float_converter)
df_risultati_elezioni_presidenziali_1_turno = df_risultati_elezioni_presidenziali_1_turno.astype({"codice_regione": "Int64", "codice_EPCI": "Int64", "codice_circoscrizione": "Int64", "% voti/espressi": np.float64}, errors='ignore')
df_risultati_elezioni_presidenziali_1_turno = df_risultati_elezioni_presidenziali_1_turno.astype({"codice_regione": str, "codice_EPCI": str, "codice_circoscrizione": str})
df_risultati_elezioni_presidenziali_1_turno.drop(['tendenza_politica'], axis=1, inplace=True)
df_risultati_elezioni_presidenziali_1_turno.to_csv(file_path_risultati_elezioni_presidenziali_1_turno, index=False)
df_risultati_elezioni_presidenziali_1_turno.head(10)

AttributeError: 'float' object has no attribute 'replace'

In [ ]:
# Caricamento e cleaning dataset risultati_elezioni_presidenziali_2_turno

df_risultati_elezioni_presidenziali_2_turno = pd.read_csv(file_path_risultati_delle_elezioni_presidenziali_2_turno, sep=';', dtype={"Code du département": str, "Code de la commune": str, "Code du b.vote": str})
df_risultati_elezioni_presidenziali_2_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento", "Code de la circonscription" : "codice_circoscrizione", "Libellé de la circonscription" : "nome_circoscrizione", "Code de la commune" : "codice_comune", "Libellé de la commune" : "nome_comune", "Code du b.vote" : "codice_seggio_elettorale", "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti", "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti", "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti", "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti", "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome", "Nuance" : "tendenza_politica", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/espressi", "Code Officiel EPCI" : "codice_EPCI", "Nom Officiel EPCI" : "nome_EPCI", "Code Officiel Région" : "codice_regione", "Nom Officiel Région" : "nome_regione", "location" : "coordinate", "lib_du_b_vote" : "etichetta_seggio_elettorale", "scrutin_code" : "codice_scrutinio"}, inplace=True)
new_ordered_columns = ["codice_regione","nome_regione","codice_dipartimento","nome_dipartimento","codice_EPCI", "nome_EPCI","codice_comune","nome_comune","codice_circoscrizione","nome_circoscrizione","codice_seggio_elettorale","iscritti","astenuti","% astenuti/iscritti","votanti","% votanti/iscritti","bianche","% bianche/iscritti","% bianche/votanti","nulle","% nulle/iscritti","% nulle/votanti","espressi","% espressi/iscritti","% espressi/votanti","pannello_numero","genere","cognome","nome","tendenza_politica","voti","% voti/iscritti","% voti/espressi","coordinate","etichetta_seggio_elettorale","codice_scrutinio"]
df_risultati_elezioni_presidenziali_2_turno = df_risultati_elezioni_presidenziali_2_turno.reindex(columns=new_ordered_columns)
df_risultati_elezioni_presidenziali_2_turno['% voti/espressi'] = df_risultati_elezioni_presidenziali_2_turno['% voti/espressi'].map(float_converter)
df_risultati_elezioni_presidenziali_2_turno = df_risultati_elezioni_presidenziali_2_turno.astype({"codice_regione": "Int64", "codice_EPCI": "Int64", "codice_circoscrizione": "Int64", "% voti/espressi": np.float64}, errors='ignore')
df_risultati_elezioni_presidenziali_2_turno = df_risultati_elezioni_presidenziali_2_turno.astype({"codice_regione": str, "codice_EPCI": str, "codice_circoscrizione": str})
df_risultati_elezioni_presidenziali_2_turno.drop(['tendenza_politica'], axis=1, inplace=True)
df_risultati_elezioni_presidenziali_2_turno.to_csv(file_path_risultati_elezioni_presidenziali_2_turno, index=False)
df_risultati_elezioni_presidenziali_2_turno.head(10)

In [ ]:
# Caricamento e cleaning dataset disoccupazione

df_disoccupazione = pd.read_csv(file_path_dati_sulla_disoccupazione, sep=';')
df_disoccupazione.rename(columns={"Période (Trimestre)": "periodo_trimestre","Date":"data","Catégorie":"categoria","Tranche d'âge":"fascia_eta","Nb moyen demandeur emploi":"numero_medio_persone_cercanti_lavoro","Nb moyen demandeur emploi Homme":"numero_medio_uomini_cercanti_lavoro","Nb moyen demandeur emploi Femme":"numero_medio_donne_cercanti_lavoro","Différence Nb demandeur emploi trimestre précédent":"differenza_con_cercanti_lavoro_trimestre_prec","Nb moyen entrées":"numero_medio_entrate","Nb moyen sorties":"numero_medio_uscite","Nb moyen demandeur emploi longue durée":"numero_medio_persone_cercanti_lavoro_lungo_termine","Code Officiel Département":"codice_dipartimento","Code Officiel Région":"codice_regione","Nom Officiel Région":"nome_regione","Code Iso 3166-3 Zone":"Code Iso 3166-3 Zona","Nom Officiel Département":"nome_dipartimento"}, inplace=True)
new_ordered_columns = ["periodo_trimestre","data","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","categoria","fascia_eta", "numero_medio_persone_cercanti_lavoro","numero_medio_persone_cercanti_lavoro_lungo_termine","numero_medio_uomini_cercanti_lavoro","numero_medio_donne_cercanti_lavoro","differenza_con_cercanti_lavoro_trimestre_prec","numero_medio_entrate","numero_medio_uscite","Code Iso 3166-3 Zona"]
df_disoccupazione = df_disoccupazione.reindex(columns=new_ordered_columns)
df_disoccupazione.drop(['Code Iso 3166-3 Zona', 'periodo_trimestre'], axis=1, inplace=True)
#df_disoccupazione.head(10)

In [ ]:
# Caricamento e cleaning dataset andamento_prezzi_carburanti

df_andamento_prezzi_carburanti = pd.read_csv(file_path_andamento_dei_prezzi_dei_carburanti, sep=';', dtype={"Code officiel commune ou arrondissement": str, "Nom Officiel Département": str, "Code Officiel Département": str})
df_andamento_prezzi_carburanti.rename(columns={"Timetable":"orario","Identifiant":"identificativo","Code Postal":"cap","Présence":"presenza","Adresse":"indirizzo","Nom Officiel Commune / Arrondissement Municipal Majuscule":"nome_comune","Code officiel commune ou arrondissement":"codice_comune","Nom Officiel Département":"nome_dipartimento","Code Officiel Département":"codice_dipartimento","Nom Officiel Région":"nome_regione","Code Officiel Région":"codice_regione","Nom Officiel EPCI":"nome_EPCI","Code Officiel EPCI":"codice_EPCI","Automate 24-24":"automatico","Carburant":"carburante","Rupture":"interruzione","Mise à jour":"aggiornamento","Prix Gazole":"prezzo_diesel","Prix SP95":"prezzo_SP95","Prix SP98":"prezzo_SP98","Prix GPLc":"prezzo_GPLc","Prix E10":"prezzo_E10","Prix E85":"prezzo_E85","Services":"servizi","Marque":"marca","Nom":"nome","Geo Point":"punto_geografico"}, inplace=True)
new_ordered_columns = ["orario","identificativo","cap","presenza","indirizzo","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","nome_EPCI","codice_EPCI","nome_comune","codice_comune","automatico","carburante","interruzione","aggiornamento","prezzo_diesel","prezzo_SP95","prezzo_SP98","prezzo_GPLc","prezzo_E10","prezzo_E85","servizi","marca","nome","punto_geografico"]
df_andamento_prezzi_carburanti = df_andamento_prezzi_carburanti.reindex(columns=new_ordered_columns)
#df_andamento_prezzi_carburanti.head(10)

In [ ]:
# Query n.1

# • I 20 dipartimenti con la maggiore percentuale di voti per Macron
#     • Primo e secondo turno

mask = (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Hauts-de-France') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Grand Est') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Île-de-France') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Normandie') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Bretagne') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Bourgogne-Franche-Comté') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Centre-Val de Loire')
df_risultati_elezioni_presidenziali_1_turno = df_risultati_elezioni_presidenziali_1_turno[mask]
df_risultati_elezioni_presidenziali_1_turno.reset_index(drop=True, inplace=True)

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='MACRON']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_1_turno = df_final[:20]
display(df_query_1_turno)
lista_dipartimenti_1_turno = df_query_1_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_1_turno)



mask = (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Hauts-de-France') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Grand Est') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Île-de-France') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Normandie') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Bretagne') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Bourgogne-Franche-Comté') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Centre-Val de Loire')
df_risultati_elezioni_presidenziali_2_turno = df_risultati_elezioni_presidenziali_2_turno[mask]
df_risultati_elezioni_presidenziali_2_turno.reset_index(drop=True, inplace=True)

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_2_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='MACRON']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_2_turno = df_final[:20]
display(df_query_2_turno)
lista_dipartimenti_2_turno = df_query_2_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_2_turno)

In [ ]:
# Query n.2

# I 20 dipartimenti con la minore percentuale di voti per Macron
#     • Primo e secondo turno

mask = (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Hauts-de-France') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Grand Est') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Île-de-France') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Normandie') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Bretagne') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Bourgogne-Franche-Comté') | \
       (df_risultati_elezioni_presidenziali_1_turno['nome_regione'] == 'Centre-Val de Loire')
df_risultati_elezioni_presidenziali_1_turno = df_risultati_elezioni_presidenziali_1_turno[mask]
df_risultati_elezioni_presidenziali_1_turno.reset_index(drop=True, inplace=True)

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='MACRON']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_1_turno = df_final[-20:]
display(df_query_1_turno)
lista_dipartimenti_1_turno = df_query_1_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_1_turno)



mask = (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Hauts-de-France') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Grand Est') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Île-de-France') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Normandie') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Bretagne') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Bourgogne-Franche-Comté') | \
       (df_risultati_elezioni_presidenziali_2_turno['nome_regione'] == 'Centre-Val de Loire')
df_risultati_elezioni_presidenziali_2_turno = df_risultati_elezioni_presidenziali_2_turno[mask]
df_risultati_elezioni_presidenziali_2_turno.reset_index(drop=True, inplace=True)

# Raggruppa i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiene la somma dei valori della colonna 'voti' e della colonna 'espressi'
df_final = df_risultati_elezioni_presidenziali_2_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
df_final = df_final[df_final['cognome']=='MACRON']
df_final['% voti_a_favore/voti_totali'] = df_final.apply(percentage_calculator, axis=1)
df_final = df_final.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
df_final.reset_index(drop=True, inplace=True)
df_query_2_turno = df_final[-20:]
display(df_query_2_turno)
lista_dipartimenti_2_turno = df_query_2_turno["nome_dipartimento"].values.tolist()
display(lista_dipartimenti_2_turno)

In [ ]:
# Query n.3

In [ ]:
# Query n.4

In [ ]:
# Query n.5

In [ ]:
# Query n.6
#I 20 dipartimenti con la maggiore disoccupazione

mask = (df_disoccupazione['nome_regione'] == 'Hauts-de-France') | \
       (df_disoccupazione['nome_regione'] == 'Grand Est') | \
       (df_disoccupazione['nome_regione'] == 'Île-de-France') | \
       (df_disoccupazione['nome_regione'] == 'Normandie') | \
       (df_disoccupazione['nome_regione'] == 'Bretagne') | \
       (df_disoccupazione['nome_regione'] == 'Bourgogne-Franche-Comté') | \
       (df_disoccupazione['nome_regione'] == 'Centre-Val de Loire')
df_disoccupazione = df_disoccupazione[mask]
df_disoccupazione.reset_index(drop=True, inplace=True)

#seleziono i dati riguardanti il 2022
dati_2022 = df_disoccupazione[pd.to_datetime(df_disoccupazione['data']).dt.year == 2022]

#definisco una funzione per ottenere i primi 20 dipartimenti con la maggior disoccupazione per ciascun gruppo
def get_top_20_departments(group):
    return group.nlargest(20, 'numero_medio_persone_cercanti_lavoro')

# Calcolo i primi 20 dipartimenti con la maggior disoccupazione per ciascun trimestre del 2022
df_final = dati_2022.groupby('data').apply(get_top_20_departments).reset_index(drop=True)

display(df_final)

In [ ]:
# Query n.7
#I 20 dipartimenti con la minore disoccupazione

mask = (df_disoccupazione['nome_regione'] == 'Hauts-de-France') | \
       (df_disoccupazione['nome_regione'] == 'Grand Est') | \
       (df_disoccupazione['nome_regione'] == 'Île-de-France') | \
       (df_disoccupazione['nome_regione'] == 'Normandie') | \
       (df_disoccupazione['nome_regione'] == 'Bretagne') | \
       (df_disoccupazione['nome_regione'] == 'Bourgogne-Franche-Comté') | \
       (df_disoccupazione['nome_regione'] == 'Centre-Val de Loire')
df_disoccupazione = df_disoccupazione[mask]
df_disoccupazione.reset_index(drop=True, inplace=True)

#seleziono i dati riguardanti il 2022
dati_2022 = df_disoccupazione[pd.to_datetime(df_disoccupazione['data']).dt.year == 2022]

#definisco una funzione per ottenere i primi 20 dipartimenti con la minor disoccupazione per ciascun gruppo
def get_bottom_20_departments(group):
    return group.nsmallest(20, 'numero_medio_persone_cercanti_lavoro')

# Calcolo i primi 20 dipartimenti con la minor disoccupazione per ciascun trimestre del 2022
df_final = dati_2022.groupby('data').apply(get_bottom_20_departments).reset_index(drop=True)

display(df_final)

In [ ]:
# Query n.8

In [ ]:
# Query n.9

In [ ]:
# Query n.10

In [ ]:
# Query n.11

In [ ]:
# Query n.12

In [ ]:
# Query n.13

In [ ]:
# Query n.14